In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

# How to extract, fit, center and wrap to MIC a trajectory

In [3]:
trajectory = msm.demo['pentarginine']['traj.h5']

In [ ]:
msm.info(trajectory)

In [ ]:
msm.view(trajectory, standard=True, with_water_as='licorice')

## Extract

## Using the method convert

In [ ]:
msm.convert(trajectory, to_form='last_10_frames.h5', structure_indices=range(89,99))

In [ ]:
msm.info('last_10_frames.h5')

In [ ]:
msm.convert(trajectory, to_form='traj_ions_peptide.h5', selection='molecule_type!="water"')

In [ ]:
msm.info('traj_ions_peptide.h5')

### Using an iterator

In [4]:
iterator = msm.Iterator(trajectory)

/home/diego/projects@dprada/MolSysMT/molsysmt/basic/iterator.py:69: NotDigestedArgumentWarning: The output from molsysmt.form.file_h5.iterators.__init__ argument was not digested.Check  for more information. If you still need help, open a new issue in https://github.com/uibcdf/MolSysMT/issues.
  tmp_iterator = dict_structures_iterator[aux_items_forms[item]](item, atom_indices=self.atom_indices, start=self.start,
/home/diego/projects@dprada/MolSysMT/molsysmt/form/file_h5/iterators.py:15: NotDigestedArgumentWarning: The output from molsysmt.form.mdtraj_HDF5TrajectoryFile.iterators.__init__ argument was not digested.Check  for more information. If you still need help, open a new issue in https://github.com/uibcdf/MolSysMT/issues.
  super().__init__(molecular_system, atom_indices=atom_indices, start=start, step=step, stop=stop,


In [ ]:
iterator = msm.Iterator(trajectory, structure_indices=range(89,99))

for molecular_system in iterator:
    msm.append_structure('last_10_frames.h5', molecular_system)

Or

In [ ]:
iterator = msm.Iterator(trajectory, selection='molecule_type!="water"')

for molecular_system in iterator:
    msm.append_structure('traj_ions_peptide.h5', molecular_system)

## Fit, center, and wrap to MIC

### Working with a molecular system in memory

In [ ]:
molecular_system = msm.convert(trajectory)

In [ ]:
molecular_system = msm.structure.fit(molecular_system, selection='atom_name in ["CA", "C", "N"]')
molecular_system = msm.structure.center(molecular_system, center_of_selection='atom_name in ["CA", "C", "N"]')
molecular_system = msm.pbc.wrap_to_mic(molecular_system)

In [ ]:
msm.info(molecular_system)

In [ ]:
msm.view(molecular_system, standard=True, water_as_surface=True)

With an interator and a molecular system in memory

### Using an iterator

In [ ]:
msm.file.open()

In [ ]:
msm.file.get_status()

In [ ]:
msm.file.close()

In [ ]:
iterator = msm.Iterator(trajectory)

In [ ]:
for molecular_system in iterator:
    molecular_system = msm.structure.fit(aux_molecular_system, selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.structure.center(molecular_system, center_of_selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.pbc.wrap_to_mic(molecular_system)
    msm.append_structures('asdf.h5', molecular_system)

In [ ]:
msm.info(final_molecular_system)

In [ ]:
msm.view(final_molecular_system, standard=True, water_as_surface=True)

There is no need to work with an iterator and a molsysmt.MolSys object. You can explore other possilibities like iterating only the coordinates and boxes:

In [ ]:
topology = msm.convert('last_100_frames.h5', 'molsysmt.Topology')

all_coordinates = []
all_boxes = []

iterator = msm.Iterator('last_100_frames.h5', coordinates=True, box=True)

for coordinates, box in iterator:
    molecular_system = [topology, coordinates, box]
    molecular_system = msm.structure.fit(molecular_system, selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.structure.center(molecular_system, center_of_selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.pbc.wrap_to_mic(molecular_system)
    new_coordinates = msm.get(molecular_system, coordinates=True) # new_coordinates = molecular_system[1]
    all_coordinates.append(new_coordinates)
    all_box.append(box)

In [ ]:
msm.info(molecular_system)

In [ ]:
msm.view(molecular_system, standard=True, water_as_surface=True)

Or:

In [ ]:
atoms_to_fit_and_center = msm.select('last_100_frames.h5', selection='atom_name in ["CA", "C", "N"]')

all_coordinates = []
all_boxes = []

iterator = msm.Iterator('last_100_frames.h5', coordinates=True, box=True)

for coordinates, box in iterator:
    coordinates = msm.structure.fit(coordinates, selection=atoms_to_fit_and_center)
    coordinates = msm.structure.center(coordinates, center_of_selection=atoms_to_fit_and_center)
    coordinates, _ = msm.pbc.wrap_to_mic([coordinates, box])
    all_coordinates.append(coordinates)
    all_boxes.append(box)

In [ ]:
msm.view(['last_100_frames.h5', coordinates, boxes], standard=True, water_as_surface=True)

################
2 notebooks in the cookbook:
    - writing a trajectory file
    - analysing a trajectory file